In [14]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv
from urllib.parse import urljoin
 
def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?p={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        
        if response.status_code == 429:
            print(f"Trop de requêtes, en pause pendant un moment...")
            time.sleep(2)  # Pause de 10 secondes en cas d'erreur 429
            return get_car_links(main_url, page_number)  # Réessayer après la pause
        
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        base_url = "https://www.avito.ru/all/avtomobili"  # Ajoutez le domaine de base ici
        car_links = [urljoin(base_url, a['href']) for a in soup.select('a.styles-module-root-QmppR styles-module-root_noVisited-aFA10')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez le nom de la voiture
        car_name_element = car_soup.select_one('.styles-module-root-TWVKW.styles-module-root-_KFFt.styles-module-size_xxxl-A2qfi.styles-module-size_xxxl-_bK04.stylesMarningNormal-module-root-OSCNq.stylesMarningNormal-module-header-3xl-k0ckc')
        car_name = car_name_element.get_text(strip=True) if car_name_element else "Nom non trouvé"

        # Obtenez les détails de la voiture
        car_details_elements = car_soup.select('.item-view .item-params')  # Mise à jour du sélecteur
        car_features = {}

        for detail_element in car_details_elements:
            feature_icon = detail_element.select_one('.icones')
            feature_name = detail_element.select_one('p').get_text(strip=True) if detail_element.select_one('p') else "Nom de la caractéristique non trouvé"

            if feature_icon:
                feature_class = feature_icon.get('class')[1]
                car_features[feature_class] = feature_name

        # Ajoutez du code pour extraire le prix de la voiture
        car_price_element = car_soup.select_one('.styles-module-size_xxxl-A2qfi')
        car_price = car_price_element.get_text(strip=True) if car_price_element else "Prix non trouvé"

        print(f"Nom de la voiture: {car_name}")
        print(f"Caractéristiques de la voiture:\n{car_features}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Name': car_name, 'Car Features': car_features, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

total_pages = 2  # Modifier le nombre total de pages selon les besoins
car_details_list = []

for page_number in range(1, total_pages + 1):
    main_page_url = 'https://www.avito.ru/all/avtomobili'
    car_links = get_car_links(main_page_url, page_number)

    for car_link in car_links:
        print(f"Scraping details for car: {car_link}")
        car_detail = scrape_car_details(car_link)
        
        if car_detail:
            car_details_list.append(car_detail)
        else:
            print(f"Aucun détail trouvé pour la voiture : {car_link}")

        time.sleep(2)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details_avito.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Features', 'Car Price']  # Ajoutez 'Car Price' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Trop de requêtes, en pause pendant un moment...


KeyboardInterrupt: 

In [20]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?p={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        car_links = [a['href'] for a in soup.select('a._blank')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez le nom de la voiture
        car_name_element = car_soup.find('span', class_='clad_name')
        car_name = car_name_element.text.strip() if car_name_element else "Nom non trouvé"

        # Obtenez les détails de la voiture
        car_details_elements = car_soup.find('ul', class_='auto_params')
        car_details = '\n'.join(detail.text.strip() for detail in car_details_elements.find_all('li')) if car_details_elements else "Details non trouvé"

        # Obtenez le prix de la voiture
        car_price_element = car_soup.find('span', class_='js-item-price')
        car_price = car_price_element.text.strip() if car_price_element else "Prix non trouvé"

        # Processus des données récupérées
        print(f"Nom de la voiture: {car_name}")
        print(f"Details de la voiture:\n{car_details}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Name': car_name, 'Car Details': car_details, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

def scrap_pages(main_url, start_page, end_page):
    car_details_list = []

    for page_number in range(start_page, end_page + 1):
        car_links = get_car_links(main_url, page_number)

        for car_link in car_links:
            print(f"Scrapping details for car: {car_link}")
            car_detail = scrape_car_details(car_link)
            
            if car_detail:
                car_details_list.append(car_detail)
            else:
                print(f"Aucun détail trouvé pour la voiture : {car_link}")
            
            time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

    return car_details_list

main_page_url = 'https://www.avito.ru/all/avtomobili'
total_pages = 2  # Modifier le nombre total de pages selon les besoins
start_page = 1
end_page = total_pages
car_details_list = scrap_pages(main_page_url, start_page, end_page)

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details_avito.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Details', 'Car Price']  # Ajoutez 'Car Name' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Erreur lors de la récupération des liens de la page 2 : 429 Client Error: Too Many Requests for url: https://www.avito.ru/all/avtomobili?p=2
Nombre total de détails récupérés : 0
